In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from numpy.linalg import norm

In [2]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data',header=None)
scaler = MinMaxScaler()
df = scaler.fit_transform(df)
df = pd.DataFrame(df)

/Users/kareem/Library/Python/2.7/lib/python/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [3]:
df.columns = ["class",
            "alcohol",
            "malic_acid",
            "ash",
            "alcalinity_of_ash",
            "magnesium",
            "total_phenols",
            "flavanoids",
            "nonflavanoid_phenols",
            "proanthocayanins",
            "color_intensity",
            "hue",
            "od280",
            "proline"]
df = df.drop(["class"],axis=1)

In [4]:
def init_whale(spaceDim,numCenters,numPopulation,lowerBound,upperBound):
    whalePop = []
    for i in range(numPopulation):
        tmp = np.random.rand(numCenters*spaceDim)*(upperBound-lowerBound)-(upperBound-lowerBound)/2
        whalePop.append(tmp)
    return whalePop

In [5]:
maxIter = 100
numPopulation = 40
spaceDim = len(df.columns)
P = len(df) #178
numCenters = 3
lowerBound = -2 #must be negativ
upperBound = 2 #must be positive
b = 1.0

In [6]:
whalePop = init_whale(spaceDim,numCenters,numPopulation,lowerBound,upperBound)

In [7]:
t = 0
count = 0
bestWhale = None

while t<maxIter:
    
    #STEP 1 ---------------------------------------
    
    fitnessPop = []
    for whale in whalePop:
        fitnessWhale = 0
        for p in range(P):
            distances = [norm(df.iloc[p]-whale[0:13]),norm(df.iloc[p]-whale[13:26]),norm(df.iloc[p]-whale[26:39])] #
            idxDist = np.argmin(distances)
            fitnessWhale += distances[idxDist]
        fitnessPop.append(fitnessWhale)
        count += 1
        idxFit = np.argmin(fitnessPop)
    bestWhale = whalePop[idxFit]
    print("Fitness of best whale "+str(fitnessPop[idxFit]))
        
    #STEP 2 ---------------------------------------
    
    for i,whale in enumerate(whalePop,0):
        
        #Update values a,A,C,l,p
        a = 2-(2*t)/maxIter #decreases
        r1 = np.random.rand(numCenters*spaceDim) #could also be a number
        r2 = np.random.rand(numCenters*spaceDim)
        A = 2*a*r1-a #decreases
        C = 2*r1
        p = np.random.uniform(0,1,1)[0]
        l = np.random.uniform(-1,1,1)[0]
        
        if p < 0.5:
            if norm(A) < 1:
                
                #Phase of encircling
                #Note that with increasing iteration exploration will stop
                D = np.abs(np.multiply(C,bestWhale)-whale)
                whalePop[i] = bestWhale-np.multiply(A,D)
                
            elif norm(A) >= 1: 
                
                #Phase of exploration
                XRand = np.random.rand(numCenters*spaceDim) #must not necessarily be in [0,1]
                D = np.abs(np.multiply(C,bestWhale)-whale)
                whalePop[i] = XRand-np.multiply(A,D)
                
        elif p>= 0.5:
            
            #Phase of spiral
            D_ = np.abs(bestWhale-whale)
            whalePop[i] = D_*np.exp(b*l)*np.cos(2*np.pi*l)+bestWhale
            
    t += 1
    print("Iteration "+str(t))

Fitness of best whale 472.4384935781959
Iteration 1
Fitness of best whale 409.0805453596742
Iteration 2
Fitness of best whale 324.80259751156177
Iteration 3
Fitness of best whale 324.80259751156177
Iteration 4
Fitness of best whale 276.71096562603884
Iteration 5
Fitness of best whale 241.04395167016733
Iteration 6
Fitness of best whale 238.62700318470354
Iteration 7
Fitness of best whale 232.6495814344543
Iteration 8
Fitness of best whale 232.582308119922
Iteration 9
Fitness of best whale 231.29516473763522
Iteration 10
Fitness of best whale 228.69923233530886
Iteration 11
Fitness of best whale 216.67131601103077
Iteration 12
Fitness of best whale 215.4897044637594
Iteration 13
Fitness of best whale 201.77877520679323
Iteration 14
Fitness of best whale 196.5554285759018
Iteration 15
Fitness of best whale 194.97189388123414
Iteration 16
Fitness of best whale 193.18671425162162
Iteration 17
Fitness of best whale 187.86969951840254
Iteration 18
Fitness of best whale 187.86969951840254
Ite

In [8]:
z1,z2,z3 = bestWhale[0:13],bestWhale[13:26],bestWhale[26:39]

In [9]:
cluster = []
for p in range(P):
    distances = [norm(df.iloc[p]-z1),norm(df.iloc[p]-z3),norm(df.iloc[p]-z3)]
    cluster.append(np.argmin(distances)+1)
df["cluster"] = cluster